In [206]:
import pandas as pd
import io
import requests
import pymysql
import pyodbc
from urllib.parse import quote_plus
from sqlalchemy import create_engine, event
import numpy as np
import pyodbc
import sodapy
from sodapy import Socrata
import math

In [136]:
#url="https://data.sfgov.org/resource/wr8u-xric.csv"
#s=requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('utf-8')))

#df=pd.read_csv("https://data.sfgov.org/resource/wr8u-xric.csv", nrows = 999999)

#df.info()
for chunk in pd.read_csv("https://data.sfgov.org/resource/wr8u-xric.csv", nrows=10000, chunksize=10000):
     chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 64 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   incident_number                               1000 non-null   int64  
 1   exposure_number                               1000 non-null   int64  
 2   id                                            1000 non-null   int64  
 3   address                                       1000 non-null   object 
 4   incident_date                                 1000 non-null   object 
 5   call_number                                   1000 non-null   int64  
 6   alarm_dttm                                    1000 non-null   object 
 7   arrival_dttm                                  1000 non-null   object 
 8   close_dttm                                    1000 non-null   object 
 9   city                                          1000 non-null   ob

In [137]:
client = Socrata("data.sfgov.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("wr8u-xric", limit=500)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 64 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   incident_number                               500 non-null    object
 1   exposure_number                               500 non-null    object
 2   id                                            500 non-null    object
 3   address                                       500 non-null    object
 4   incident_date                                 500 non-null    object
 5   call_number                                   500 non-null    object
 6   alarm_dttm                                    500 non-null    object
 7   arrival_dttm                                  500 non-null    object
 8   close_dttm                                    500 non-null    object
 9   city                                          500 non-null    object
 10  zi

In [174]:
server = '<databasename>' 
database = 'datafactory_fire_poc' 
username = '<username>' 
password = '<password>' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


In [289]:
for i in results_df.index:
    
    ##Exposure Dimension
    ##print(results_df["address"][i])
    ##cursor.execute("SELECT * FROM dbo.Exposure WHERE exposureNumber = ?", results_df["point"][0])
    exposureId = upsert_exposure(results_df["exposure_number"][i])
    
    #Location Dimension
    locationId = upsert_location(results_df["address"][i], df["point"][i])
    #print(locationId)
    
    #Time Dimension
    idTime = upsert_time(results_df["incident_date"][i])
    #print(idTime)
    
    #City Dimension
    idCity = upsert_city(results_df["city"][i])
    
    #Station Dimension
    idStation = upsert_station(results_df["station_area"][i], results_df["battalion"][i])
    
    #Unit Dimension
    #idUnit = upsert_unit(results_df["station_area"][i], results_df["battalion"][i])
    
    #Situation Dimension
    situationSplitted = results_df["primary_situation"][i].split(' - ')
    #print(situationSplitted)
    idSituation = upsert_situation(situationSplitted[0], situationSplitted[1])
    
    #Action Dimension
    actionSplitted = results_df["action_taken_primary"][i].split(' - ')
    #print(actionSplitted)
    idAction = upsert_action(actionSplitted[0], actionSplitted[1])
    
    #Property Use Dimension
    propertyUseSplitted = results_df["property_use"][i].split(' - ')
    #print(actionSplitted)
    if len(propertyUseSplitted)>1:
        idPropertyUse = upsert_propertyuse(propertyUseSplitted[0], propertyUseSplitted[1])
        
    
    #Fire Area Dimension
    if pd.notnull(results_df["area_of_fire_origin"][i]):
        fireAreaSplitted = results_df["area_of_fire_origin"][i].split(' - ')
        #print(fireAreaSplitted)
        if len(fireAreaSplitted)>1:
            idFireArea= upsert_firearea(fireAreaSplitted[0], fireAreaSplitted[1])
    
    #District Dimension
    idDistrict = upsert_district(results_df["neighborhood_district"][i])
    
    #Fact Table
    upsert_facttable(results_df["incident_number"][i], results_df["id"][i], results_df["suppression_units"][i], results_df["suppression_personnel"][i], results_df["ems_units"][i], results_df["ems_personnel"][i], results_df["other_units"][i], results_df["other_personnel"][i], results_df["first_unit_on_scene"][i], results_df["estimated_property_loss"][i], results_df["estimated_contents_loss"][i], results_df["fire_fatalities"][i], results_df["fire_injuries"][i], results_df["civilian_fatalities"][i], results_df["civilian_injuries"][i], results_df["number_of_alarms"][i], results_df["mutual_aid"][i], results_df["floor_of_fire_origin"][i], results_df["number_of_floors_with_minimum_damage"][i], results_df["number_of_floors_with_significant_damage"][i], results_df["number_of_floors_with_heavy_damage"][i], results_df["number_of_floors_with_extreme_damage"][i], exposureId,locationId,idTime,idCity,idStation,idSituation,idAction,idPropertyUse,idFireArea,idDistrict)

8028304, 080283040
1
4
0
0
0
0
E11
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
2
83
1
1
1
1
1
2
1
8028303, 080283030
1
4
0
0
0
0
E37
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
3
83
1
2
2
2
2
2
2
8028309, 080283090
10
35
0
0
0
0
E01
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
4
83
1
3
3
1
3
2
3
8028314, 080283140
1
4
0
0
0
0
E36
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
5
83
1
4
4
3
4
2
4
8028319, 080283190
1
4
0
0
0
0
E14
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
6
83
1
5
5
4
2
2
5
8028337, 080283370
3
10
0
0
0
0
E12
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
8
83
1
6
6
1
4
2
6
8028329, 080283290
2
9
0
0
0
0
E01
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
9
83
1
3
7
1
5
2
7
8028350, 080283500
2
9
0
0
0
0
E32
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
10
83
1
7
7
1
2
2
1
8028355, 080283550
1
4
0
0
0
0
T08
nan
nan
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
11
83
1
8
4
1
5
2
7
8028352, 080283520
3
10
0
0
0
0
E22
nan
1
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
12
83
1
9
8
1
4
1
6
80283

3007737, 030077370
2
9
0
0
0
0
E36
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
86
89
1
4
7
1
1
4
4
3007738, 030077380
1
4
0
0
0
0
E09
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
87
89
1
34
11
1
1
4
16
3007740, 030077400
3
11
0
0
0
0
E03
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
88
89
1
17
38
1
20
4
13
3007741, 030077410
3
11
0
0
0
0
E01
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
89
89
1
17
38
1
21
4
9
3007742, 030077420
2
9
0
0
0
0
E36
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
90
89
1
3
7
1
1
4
3
3007747, 030077470
2
9
0
0
0
0
nan
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
91
89
1
3
7
1
1
4
9
Oops! <class 'pyodbc.ProgrammingError'> occurred.
Next entry.

3007751, 030077510
2
9
0
0
0
0
E36
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
92
89
1
4
7
1
1
4
4
3008177, 030081770
1
4
0
0
0
0
T18
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
93
90
1
29
10
6
22
4
18
3008178, 030081780
1
4
0
0
0
0
E29
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
94
90
1
31
39
7
2
5
2
3008183, 030081830
3
11
0
0
0
0
E07
0
0
0
0
0
0
1
None


3003820, 030038200
1
4
0
0
0
0
T02
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
161
102
1
21
51
7
1
7
19
3003851, 030038510
1
4
0
0
0
0
E42
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
162
102
1
11
1
6
1
7
10
3003835, 030038350
1
4
0
0
0
0
E19
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
163
102
1
26
4
1
29
7
23
3003836, 030038360
1
4
0
0
0
0
E20
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
164
102
1
42
48
26
8
7
18
3003839, 030038390
1
4
0
0
0
0
E40
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
165
102
1
29
7
1
2
7
18
3003840, 030038400
8
30
1
2
0
0
E12
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
166
102
1
6
8
7
3
7
31
3003842, 030038420
2
9
0
0
0
0
E13
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
167
102
1
21
7
1
5
7
7
3003846, 030038460
2
9
0
0
0
0
E13
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
168
102
1
21
14
1
6
7
7
3003875, 030038750
3
11
0
0
0
0
E28
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
169
102
1
24
38
1
15
7
19
3003877, 030038770
1
4
0
0
0
0
E16
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
170
102
1
33
19
1


3008951, 030089510
1
4
0
0
0
0
nan
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
235
91
1
33
36
26
7
8
29
Oops! <class 'pyodbc.ProgrammingError'> occurred.
Next entry.

3008956, 030089560
2
9
0
0
0
0
E03
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
236
91
1
17
21
1
1
8
12
3008745, 030087450
1
4
0
0
0
0
E21
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
237
91
1
27
57
1
3
8
21
3008747, 030087470
2
9
0
0
0
0
E12
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
238
91
1
6
41
1
3
8
6
3008752, 030087520
1
4
0
0
0
0
E32
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
239
91
1
1
4
1
1
8
22
3000212, 030002120
1
4
0
0
0
0
E03
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
240
86
1
17
20
11
10
8
9
3000215, 030002150
2
9
0
0
0
0
E36
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
241
86
1
4
46
1
5
8
16
3000224, 030002240
1
4
0
0
0
0
E22
0
0
0
0
0
0
1
None
2
1
0
0
0
1
242
86
1
9
8
1
3
1
6
3000003, 030000030
1
4
0
0
0
0
E02
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
243
86
1
30
7
1
1
1
13
3000006, 030000060
0
0
1
2
0
0
91
0
0
0
0
0
0
1
None
n

3005350, 030053500
1
4
0
0
0
0
E17
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
307
104
1
20
44
7
1
9
8
3005351, 030053510
1
5
0
0
0
0
T13
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
308
104
1
21
61
16
6
9
7
3005354, 030053540
1
4
0
0
0
0
E18
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
309
104
1
32
26
1
1
9
23
3005355, 030053550
3
11
0
0
0
0
E38
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
183
104
1
38
38
1
3
9
29
3005357, 030053570
2
4
0
0
0
0
E16
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
310
104
1
33
40
1
7
9
27
3005359, 030053590
1
4
0
0
0
0
E06
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
311
104
1
19
44
1
1
9
4
3005126, 030051260
3
11
0
0
0
0
E01
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
312
104
1
3
8
1
4
9
3
3005131, 030051310
7
26
0
0
0
0
E14
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
313
104
1
5
38
1
3
9
5
3005133, 030051330
1
5
0
0
0
0
T03
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
314
104
1
35
10
5
18
9
13
3005693, 030056930
1
4
0
0
0
0
nan
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
315
105
1
25
21
1


3000887, 030008870
3
11
0
0
0
0
E13
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
371
94
1
21
14
31
6
4
7
3000890, 030008900
1
4
0
0
0
0
nan
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
372
94
1
2
7
1
2
4
2
Oops! <class 'pyodbc.ProgrammingError'> occurred.
Next entry.

3000897, 030008970
2
11
0
0
0
0
E09
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
373
94
1
34
7
1
2
4
2
3000898, 030008980
2
9
0
0
0
0
E25
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
374
94
1
10
7
1
1
4
8
3000903, 030009030
3
11
0
0
0
0
E22
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
375
94
1
9
7
1
2
4
38
3001392, 030013920
1
3
0
0
0
0
E38
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
376
107
1
38
21
1
3
4
29
3001394, 030013940
1
4
0
0
0
0
E36
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
218
107
1
4
14
1
5
4
16
3001399, 030013990
3
11
0
0
0
0
E01
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
377
107
1
3
38
1
6
4
7
3001400, 030014000
2
6
0
0
0
0
E17
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
378
107
1
20
44
1
1
4
8
3001631, 030016310
3
11
2
3
0
0
E05
0
0
0


3006501, 030065010
3
11
0
0
0
0
nan
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
444
109
1
8
14
1
18
3
3
Oops! <class 'pyodbc.ProgrammingError'> occurred.
Next entry.

3006502, 030065020
1
4
0
0
0
0
E26
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
445
109
1
40
41
1
7
3
22
3006503, 030065030
1
4
0
0
0
0
E03
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
446
109
1
3
37
1
3
3
7
3006508, 030065080
3
11
0
0
0
0
E10
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
447
109
1
27
37
1
7
3
21
3007223, 030072230
2
9
0
0
0
0
nan
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
448
88
1
25
21
1
5
3
25
Oops! <class 'pyodbc.ProgrammingError'> occurred.
Next entry.

3007226, 030072260
3
11
0
0
0
0
E29
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
270
88
1
31
38
1
11
3
16
3007229, 030072290
7
30
0
0
0
0
E05
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
449
88
1
16
21
1
3
3
12
3007232, 030072320
1
4
0
0
0
0
E05
0
0
0
0
0
0
1
None
nan
nan
nan
nan
nan
1
449
88
1
16
21
1
8
3
12
3007234, 030072340
1
4
0
0
0
0
E11
0
0
0
0
0
0
1
None
nan
nan
nan
na

In [233]:
#results_df.info()

In [71]:
def upsert_exposure(exposure_number):
    cursor.execute("SELECT * FROM dbo.Exposure WHERE exposureNumber = ?", str(exposure_number))
    
    data = cursor.fetchone()
    
    if data is None:
        cursor.execute("INSERT INTO dbo.Exposure (exposureNumber) values(?)", str(exposure_number))
        idExposure = cursor.fetchone()[0]
        cnxn.commit()
        
        return idExposure
    
    return data[0]
        
        

In [166]:
def upsert_location(addressLocation, pointLocation):
    cursorLocation = cnxn.cursor()
    cursorLocation2 = cnxn.cursor()
    cursorLocation.execute("SELECT * FROM dbo.Location WHERE AddressLocation = ?", addressLocation)
    
    data = cursorLocation.fetchone()
    
    #print(addressLocation + " "+ pointLocation)
    
    
    if data is None and addressLocation is not None and pointLocation is not None:
        cursorLocation2.execute("INSERT INTO dbo.Location (AddressLocation, PointLocation) values(?,?)", addressLocation, pointLocation)
        
        #idLocation = 0
        cnxn.commit()
        
        cursorLocation.execute("SELECT * FROM dbo.Location WHERE AddressLocation = ?", addressLocation)
        idLocation = cursorLocation.fetchone()[0]
        
        cursorLocation2.close()
        cursorLocation.close()
        
        return idLocation
    
    return data[0]

In [73]:
def upsert_time(incidentDate):
    cursorTime = cnxn.cursor()
    cursorTime2 = cnxn.cursor()
    
    cursorTime.execute("SELECT * FROM dbo.TimeDim WHERE incidentDate = ?", incidentDate)
    #print(incidentDate)
    
    data = cursorTime.fetchone()
    
    
    if data is None:
        cursorTime2.execute("INSERT INTO dbo.TimeDim (incidentDate) values(?)", incidentDate)
        #print("inserting time: " + incidentDate)
            
        cnxn.commit()
        
        cursorTime.execute("SELECT * FROM dbo.TimeDim WHERE incidentDate = ?", incidentDate)
        idTime = cursorTime.fetchone()[0]
        
        cursorTime2.close()
        cursorTime.close()
        
        return idTime
    
    return data[0]

In [95]:
def upsert_city(city):
    cursorCity = cnxn.cursor()
    cursorCity2 = cnxn.cursor()
    
    cursorCity.execute("SELECT * FROM dbo.City WHERE CityName = ?", city)
    
    data = cursorCity.fetchone()
    cursorCity.close()
    
    #print(addressLocation + " "+ pointLocation)
    
    
    if data is None and city is not None:
        cursorCity2.execute("INSERT INTO dbo.City (CityName) values(?)", city)
        
        cnxn.commit()
        
        cursorCity2.execute("SELECT * FROM dbo.City WHERE CityName = ?", city)
        idCity = cursorCity2.fetchone()[0]
        
        cursorCity2.close()
        
        return idCity
    
    return data[0]

In [98]:
def upsert_station(stationArea, battalionNumber):
    cursorStation = cnxn.cursor()
    cursorStation2 = cnxn.cursor()
    
    cursorStation.execute("SELECT * FROM dbo.Station WHERE StationArea = ?", stationArea)
    
    data = cursorStation.fetchone()
    cursorStation.close()
    
    if data is None and stationArea is not None:
        cursorStation2.execute("INSERT INTO dbo.Station (StationArea, BattalionNumber) values(?,?)", stationArea, battalionNumber)
        
        cnxn.commit()
        
        cursorStation2.execute("SELECT * FROM dbo.Station WHERE StationArea = ?", stationArea)
        idStation = cursorStation2.fetchone()[0]
        
        cursorStation2.close()
        
        return idStation
    
    return data[0]

In [108]:
def upsert_situation(situationCode, situationDescription):
    cursorSituation = cnxn.cursor()
    cursorSituation2 = cnxn.cursor()
    
    cursorSituation.execute("SELECT * FROM dbo.Situation WHERE SituationCode = ?", situationCode)
    
    data = cursorSituation.fetchone()
    cursorSituation.close()
    if data is None and situationCode is not None and situationDescription is not None:
        cursorSituation2.execute("INSERT INTO dbo.Situation (SituationCode, SituationDescription) values(?,?)", situationCode, situationDescription)
        
        cnxn.commit()
        
        cursorSituation2.execute("SELECT * FROM dbo.Situation WHERE SituationCode = ?", situationCode)
        idSituation = cursorSituation2.fetchone()[0]
        
        cursorSituation2.close()
        
        return idSituation
    
    return data[0]

In [113]:
def upsert_action(actionCode, actionDescription):
    cursorAction = cnxn.cursor()
    cursorAction2 = cnxn.cursor()
    
    cursorAction.execute("SELECT * FROM dbo.Action WHERE ActionCode = ?", actionCode)
    
    data = cursorAction.fetchone()
    cursorAction.close()
    if data is None and actionCode is not None and actionDescription is not None:
        cursorAction2.execute("INSERT INTO dbo.Action (ActionCode, ActionDescription) values(?,?)", actionCode, actionDescription)
        
        cnxn.commit()
        
        cursorAction2.execute("SELECT * FROM dbo.Action WHERE ActionCode = ?", actionCode)
        idAction = cursorAction2.fetchone()[0]
        
        cursorAction2.close()
        
        return idAction
    
    return data[0]

In [120]:
def upsert_propertyuse(propertyUseCode, propertyUseDescription):
    cursorPropertyUse = cnxn.cursor()
    cursorPropertyUse2 = cnxn.cursor()
    
    cursorPropertyUse.execute("SELECT * FROM dbo.PropertyUse WHERE PropertyUseCode = ?", propertyUseCode)
    
    data = cursorPropertyUse.fetchone()
    cursorPropertyUse.close()
    if data is None and propertyUseCode is not None and propertyUseDescription is not None:
        cursorPropertyUse2.execute("INSERT INTO dbo.PropertyUse (PropertyUseCode, PropertyUseDescription) values(?,?)", propertyUseCode, propertyUseDescription)
        
        cnxn.commit()
        
        cursorPropertyUse2.execute("SELECT * FROM dbo.PropertyUse WHERE PropertyUseCode = ?", propertyUseCode)
        idPropertyUse = cursorPropertyUse2.fetchone()[0]
        
        cursorPropertyUse2.close()
        
        return idPropertyUse
    
    return data[0]

In [133]:
def upsert_firearea(fireAreaCode, fireAreaDescription):
    cursorFireArea = cnxn.cursor()
    cursorFireArea2 = cnxn.cursor()
    
    cursorFireArea.execute("SELECT * FROM dbo.FireArea WHERE fireAreaCode = ?", fireAreaCode)
    
    data = cursorFireArea.fetchone()
    cursorFireArea.close()
    if data is None and fireAreaCode is not None and fireAreaDescription is not None:
        cursorFireArea2.execute("INSERT INTO dbo.FireArea (FireAreaCode, FireAreaDescription) values(?,?)", fireAreaCode, fireAreaDescription)
        
        cnxn.commit()
        
        cursorFireArea2.execute("SELECT * FROM dbo.FireArea WHERE FireAreaCode = ?", fireAreaCode)
        idFireArea = cursorFireArea2.fetchone()[0]
        
        cursorFireArea2.close()
        
        return idFireArea
    
    return data[0]

In [146]:
def upsert_district(districtDescription):
    cursorDistrict = cnxn.cursor()
    cursorDistrict2 = cnxn.cursor()
    
    cursorDistrict.execute("SELECT * FROM dbo.District WHERE DistrictDescription = ?", districtDescription)
    
    data = cursorDistrict.fetchone()
    cursorDistrict.close()
    if data is None and districtDescription is not None:
        cursorDistrict2.execute("INSERT INTO dbo.District (DistrictDescription) values(?)", districtDescription)
        
        cnxn.commit()
        
        cursorDistrict2.execute("SELECT * FROM dbo.District WHERE DistrictDescription = ?", districtDescription)
        idDistrict = cursorDistrict2.fetchone()[0]
        
        cursorDistrict2.close()
        
        return idDistrict
    
    return data[0]

In [288]:
def upsert_facttable(incidentNumber,idIncident,suppressionUnits,suppresionPersonnel,emsUnits,emsPersonnel,otherUnits,otherPersonnel, firstUnit, estimatedPropertyLoss, estimatedContentLoss, fireFatalities, fireInjuries, civilianFatalities, civilianInjuries, numberOfAlarms, mutualAid, floorOfFireOrigin, numFloorsMinimumDamage, numFloorsSignificantDamage, numFloorsHeavyDamage, numFloorsExtremeDamage, idExposure,locationId,idTime,idCity,idStation,idSituation,idAction,idPropertyUse,idFireArea,idDistrict):
    cursorFire = cnxn.cursor()
    cursorFire2 = cnxn.cursor()
    
    cursorFire.execute("SELECT * FROM dbo.Fire WHERE IncidentNumber = ?", incidentNumber)
    
    print(str(incidentNumber) + ", " + str(idIncident))
    
    data = cursorFire.fetchone()
    
    
    print(suppressionUnits)
    if(np.isnan(float(suppressionUnits)) or suppressionUnits==""):
        suppressionUnits = 0
    
    print(suppresionPersonnel)
    if(np.isnan(float(suppresionPersonnel)) or suppresionPersonnel==""):
        suppresionPersonnel = 0
     
    print(emsUnits)
    if(np.isnan(float(emsUnits)) or emsUnits==""):
        emsUnits = 0
        
    print(emsPersonnel)
    if(np.isnan(float(emsPersonnel)) or emsPersonnel==""):
        emsPersonnel = 0
        
    print(otherUnits)
    if(np.isnan(float(otherUnits)) or otherUnits==""):
        otherUnits = 0
        
    print(otherPersonnel)
    if(otherPersonnel=="nan" or otherPersonnel==""):
        otherPersonnel = 0
    
    print(firstUnit)
    if(firstUnit is None or firstUnit==""):
        firstUnit = ""
        
    print(estimatedPropertyLoss)
    if(np.isnan(float(estimatedPropertyLoss)) or estimatedPropertyLoss==""):
        estimatedPropertyLoss = 0
        
    print(estimatedContentLoss)
    if(np.isnan(float(estimatedContentLoss)) or estimatedContentLoss==""):
        estimatedContentLoss = 0
    
    print(fireFatalities)
    if(np.isnan(float(fireFatalities)) or fireFatalities==""):
        fireFatalities = 0
        
    print(fireInjuries)
    if(np.isnan(float(fireInjuries)) or fireInjuries==""):
        fireInjuries = 0 
        
    print(civilianFatalities)
    if(np.isnan(float(civilianFatalities)) or civilianFatalities==""):
        civilianFatalities = 0 
    
    print(civilianInjuries)
    if(np.isnan(float(civilianInjuries)) or civilianInjuries==""):
        civilianInjuries = 0
        
    print(numberOfAlarms)
    if(np.isnan(float(numberOfAlarms)) or numberOfAlarms==""):
        numberOfAlarms = 0
    
    print(mutualAid)
    if(mutualAid is None or mutualAid==""):
        mutualAid = ""
    
    print(floorOfFireOrigin)
    if(np.isnan(float(floorOfFireOrigin)) or floorOfFireOrigin==""):
        floorOfFireOrigin = 0
        
    print(numFloorsMinimumDamage)
    if(np.isnan(float(numFloorsMinimumDamage)) or numFloorsMinimumDamage==""):
        numFloorsMinimumDamage = 0
        
    print(numFloorsSignificantDamage)
    if(np.isnan(float(numFloorsSignificantDamage)) or numFloorsSignificantDamage==""):
        numFloorsSignificantDamage = 0
        
    print(numFloorsHeavyDamage)
    if(np.isnan(float(numFloorsHeavyDamage)) or numFloorsHeavyDamage==""):
        numFloorsHeavyDamage = 0
        
    print(numFloorsExtremeDamage)
    if(np.isnan(float(numFloorsExtremeDamage)) or numFloorsExtremeDamage==""):
        numFloorsExtremeDamage  = 0
        
    print(idExposure)
    print(locationId)
    print(idTime)
    print(idCity)
    print(idStation)
    print(idSituation)
    print(idAction)
    print(idPropertyUse)
    print(idFireArea)
    print(idDistrict)
    
    cursorFire.close()
    if data is None:
        try:
            cursorFire2.execute("INSERT INTO dbo.Fire(IncidentNumber,ID,SuppressionUnits,SuppresionPersonnel, EMSUnits,EMSPersonnel, OtherUnits, OtherPersonnel, FirstUnitOnScene, EstimatedPropertyLoss,EstimatedContentLoss,FireFatalities, FireInjuries,CivilianFatalities, CivilianInjuries, NumberOfAlarms, MutualAid, FloorOfFireOrigin, NumberOfFloorsWithMinimumDamage, NumberOfFloorsWithSignificantDamage, NumberOfFloorsWithHeavyDamage, NumberOfFloorsWithExtremeDamage,idExposure,idLocation,idTime,idCity, idStation, idSituation,idAction,idPropertyUse,idFireArea,idDistrict) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", incidentNumber,idIncident,suppressionUnits,suppresionPersonnel,emsUnits,emsPersonnel,otherUnits,otherPersonnel,firstUnit,estimatedPropertyLoss,estimatedContentLoss,fireFatalities,fireInjuries,civilianFatalities,civilianInjuries,numberOfAlarms,mutualAid,floorOfFireOrigin,numFloorsMinimumDamage,numFloorsSignificantDamage,numFloorsHeavyDamage,numFloorsExtremeDamage,idExposure,locationId,idTime,idCity,idStation,idSituation,idAction,idPropertyUse,idFireArea,idDistrict)
            cnxn.commit()
        except Exception as e:
            print("Oops!", e.__class__, "occurred.")
            print("Next entry.")
            print()
            
        cursorFire2.close()

In [171]:
cnxn.close()